In [1]:
import os

from google.api_core.client_options import ClientOptions
from google.cloud.speech_v2 import SpeechClient
from google.cloud.speech_v2.types import cloud_speech

MAX_AUDIO_LENGTH_SECS = 8 * 60 * 60


def run_batch_recognize():
  # Instantiates a client.
  client = SpeechClient(
      client_options=ClientOptions(
          api_endpoint="us-central1-speech.googleapis.com",
      ),
  )

  # The output path of the transcription result.
  gcs_output_folder = "gs://test_stt_mp4_to_text/transcripts"

  # The name of the audio file to transcribe:
  audio_gcs_uri = "gs://test_stt_mp4_to_text/test_audio1.mp3"

  config = cloud_speech.RecognitionConfig(
      auto_decoding_config={},
      features=cloud_speech.RecognitionFeatures(
          enable_automatic_punctuation=True,
        ),
      model="chirp_2",
      language_codes=["en-US"],
  )

  output_config = cloud_speech.RecognitionOutputConfig(
      gcs_output_config=cloud_speech.GcsOutputConfig(uri=gcs_output_folder),
  )

  files = [cloud_speech.BatchRecognizeFileMetadata(uri=audio_gcs_uri)]

  request = cloud_speech.BatchRecognizeRequest(
      recognizer="projects/tw-maxchens-sandbox/locations/us-central1/recognizers/_",
      config=config,
      files=files,
      recognition_output_config=output_config,
  )
  operation = client.batch_recognize(request=request)

  print("Waiting for operation to complete...")
  response = operation.result(timeout=3 * MAX_AUDIO_LENGTH_SECS)
  print(response)
  
run_batch_recognize()

Waiting for operation to complete...


TimeoutError: Operation did not complete within the designated timeout of 86400 seconds.

In [23]:
! pip install --upgrade google-cloud-aiplatform


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 7.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.49.0
    Uninstalling google-cloud-aiplatform-1.49.0:
      Successfully uninstalled google-cloud-aiplatform-1.49.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
vertexai 1.49.0 requires google-cloud-aiplatform[all]==1.49.0, but you have google-cloud-aiplatform 1.62.0 which is incompatible.


### 實際完成整個 Meeting Minutes 的生成流程流程

1. 取得語音檔案：將任意檔案轉成".wav" 並且是 16000hz 的檔案
2. 調用本地端的 whisper 模型進行語音轉文字
3. 設計 Prompt 並呼叫 Gemini 模型進行會議紀錄生成

In [1]:
# 取得語音檔案本地路徑，並解析格式轉換乘 16000hz 的 ".wav" 新檔案，並記錄路徑
import os
from pydub import AudioSegment

def convert_audio_to_wav(input_file_path):
    # 獲取輸入文件的目錄和文件名（不含擴展名）
    input_dir, input_filename = os.path.split(input_file_path)
    input_name, _ = os.path.splitext(input_filename)
    
    # 創建輸出文件路徑
    output_file_path = os.path.join(input_dir, f"{input_name}_16000hz.wav")
    
    # 讀取輸入音頻文件
    audio = AudioSegment.from_file(input_file_path)
    
    # 轉換為單聲道（如果是立體聲）
    audio = audio.set_channels(1)
    
    # 設置採樣率為 16000Hz
    audio = audio.set_frame_rate(16000)
    
    # 導出為 WAV 格式
    audio.export(output_file_path, format="wav")
    
    return output_file_path

# 使用示例
input_audio_path = "/Users/mai/Downloads/0704錄音1.mp3"  # 請替換為實際的輸入音頻文件路徑 
output_wav_path = convert_audio_to_wav(input_audio_path)
print(f"轉換後的 WAV 文件路徑：{output_wav_path}")

轉換後的 WAV 文件路徑：/Users/mai/Downloads/0704錄音1_16000hz.wav


In [2]:
'''使用 Command 調用 Whisper.cpp 的模型，進行語音轉文字
Whisper.cpp 的調用方法 : 
使用步驟：
1. 先激活環境：mamba activate py310-whisper
2. 目前只接受 16000hz 的 wav : ffmpeg -i "./tests/20240402-Eupfin Bi-Weekly會議.m4a" -vn -ar 16000 -ac 1 -b:a 243k -f wav "./tests/20240402-Eupfin Bi-Weekly會議.wav" -y #如果已轉可以跳過
3. ./main -m models/ggml-large-v2.bin -l zh -otxt -f "file_path.wav" -of "file_path"'''

import subprocess
import os

# Whisper.cpp 的路徑（請根據實際情況修改）
whisper_cpp_path = "/Users/mai/Desktop/GitHub-Repos/whisper.cpp"
model_path = os.path.join(whisper_cpp_path, "models/ggml-large-v2.bin")
main_executable = os.path.join(whisper_cpp_path, "main")

# 設置輸出文件的路徑和名稱（不包括副檔名）
output_base_path = os.path.splitext(output_wav_path)[0]

# 構建 Whisper.cpp 命令
whisper_command = f"{main_executable} -m {model_path} -l zh -otxt -f {output_wav_path} -of {output_base_path}"

# 使用 conda run 來執行命令
full_command = f"conda run -n py310-whisper {whisper_command}"

try:
    # 執行命令
    result = subprocess.run(full_command, shell=True, check=True, capture_output=True, text=True)
    
    print("語音轉文字成功完成！")
    print(f"輸出文件: {output_base_path}.txt")
    
    # 如果需要，你可以讀取並打印轉錄的文本
    with open(f"{output_base_path}.txt", "r", encoding="utf-8") as f:
        transcribed_text = f.read()
        print("\n轉錄文本:")
        print(transcribed_text)

except subprocess.CalledProcessError as e:
    print(f"執行過程中發生錯誤: {e}")
    print(f"錯誤輸出: {e.stderr}")

語音轉文字成功完成！
輸出文件: /Users/mai/Downloads/0704錄音1_16000hz.txt

轉錄文本:
高中 嘉義中學
沒有啦 他是學長
學長 有一個留紙一下
小中是大一屆
大一屆 46年
他都叫他學長
那不是同學了嗎
小中就是這個都沒做
連這個你都知道
他們同年為什麼是學長
太不謙虛了
那他跟那個林董是同年
同年嘛 跟那個
蘇秀賢啊
對 所以林董也是叫那個
小中阿伯也是叫學長
叫學長 嘉義中學
是喔
好吧 我們就先開始
林董在那個地下室準備上來了
你怎麼知道
同仁有回報
那是要先開始嗎
你馬上準備上來 我在等那一張
那個沈記長 陳惠敏
也是嘉義中學的
他也是嘉義中學
嘉義 我不知道嘉義中學
我認識他 我不知道嘉義
他們都叫他
陳惠敏比較大
他是44年
他的那個氣質跟祐賢有點像
他是44年是不是
44年大兩屆
是不是嘉義中學的
嘉義 他是不是嘉義中學
好像是喔
如果不是嘉義中學
那附近有的會去台南
不然就要跑到彰化 台松
我確定他是嘉義
嘉義
應該也是嘉義中學
季老頭的腳
我那天突然發現
我們那個文化局局長
他讀的國小
跟我讀的同一個國小
高雄
我很少碰到國小的學妹
很少很少
哪一屆 你說誰
文化局局長
張嘉義
市長叫我去那個
華誼的那個
在那個中環的那個
叫我上去
他們還會見會 聊天
高雄市還是高雄縣
高雄市
來了
不好意思喔 來太慢了
沒有
還沒處理媒體事件
還沒處理好
就趕快跑來
對 最近最辛苦了
還沒處理好
那個酒店花生那個是
他也是安可經理
他媽媽不知道受到什麼刺激
不是那個 是另外一個
另外一個
那個是學校
學校端的
看他能不能辦理
好 我們如果過半了
我們就開始
新北市住宅及都市分析中心
第二屆第四次董監事會
依據本中心設置之條例
第十二條及董事會議一事規則
第四條第一項規定
本中心董事會
應經董事人士過半數次出席時的開會
本中心董事11人 監事3人
本次出席董事9人 監事2人
以符合開會人數
依主席才是予以開會
請主席致詞
我稍微更正一下
我們剛剛報告的這個
得開會的或者是有效會議的
你們有講到說我們的監察人要出席
他沒出席的話
我們都要舉牌無效會議
跟另外一邊一樣
所以我們要謝謝
我首先就是要跟大家抱歉
因為我們按照這個時程的話
我們是每兩個月一次
今年我們就是偶數月
所以應該是在6月底之前
但是議會的關係
我們原來是排在

In [1]:
with open(f"/Users/mai/Downloads/0704錄音1_16000hz.txt", "r", encoding="utf-8") as f:
        transcribed_text = f.read()
        print("\n轉錄文本:")
        print(transcribed_text)


轉錄文本:
高中 嘉義中學
沒有啦 他是學長
學長 有一個留紙一下
小中是大一屆
大一屆 46年
他都叫他學長
那不是同學了嗎
小中就是這個都沒做
連這個你都知道
他們同年為什麼是學長
太不謙虛了
那他跟那個林董是同年
同年嘛 跟那個
蘇秀賢啊
對 所以林董也是叫那個
小中阿伯也是叫學長
叫學長 嘉義中學
是喔
好吧 我們就先開始
林董在那個地下室準備上來了
你怎麼知道
同仁有回報
那是要先開始嗎
你馬上準備上來 我在等那一張
那個沈記長 陳惠敏
也是嘉義中學的
他也是嘉義中學
嘉義 我不知道嘉義中學
我認識他 我不知道嘉義
他們都叫他
陳惠敏比較大
他是44年
他的那個氣質跟祐賢有點像
他是44年是不是
44年大兩屆
是不是嘉義中學的
嘉義 他是不是嘉義中學
好像是喔
如果不是嘉義中學
那附近有的會去台南
不然就要跑到彰化 台松
我確定他是嘉義
嘉義
應該也是嘉義中學
季老頭的腳
我那天突然發現
我們那個文化局局長
他讀的國小
跟我讀的同一個國小
高雄
我很少碰到國小的學妹
很少很少
哪一屆 你說誰
文化局局長
張嘉義
市長叫我去那個
華誼的那個
在那個中環的那個
叫我上去
他們還會見會 聊天
高雄市還是高雄縣
高雄市
來了
不好意思喔 來太慢了
沒有
還沒處理媒體事件
還沒處理好
就趕快跑來
對 最近最辛苦了
還沒處理好
那個酒店花生那個是
他也是安可經理
他媽媽不知道受到什麼刺激
不是那個 是另外一個
另外一個
那個是學校
學校端的
看他能不能辦理
好 我們如果過半了
我們就開始
新北市住宅及都市分析中心
第二屆第四次董監事會
依據本中心設置之條例
第十二條及董事會議一事規則
第四條第一項規定
本中心董事會
應經董事人士過半數次出席時的開會
本中心董事11人 監事3人
本次出席董事9人 監事2人
以符合開會人數
依主席才是予以開會
請主席致詞
我稍微更正一下
我們剛剛報告的這個
得開會的或者是有效會議的
你們有講到說我們的監察人要出席
他沒出席的話
我們都要舉牌無效會議
跟另外一邊一樣
所以我們要謝謝
我首先就是要跟大家抱歉
因為我們按照這個時程的話
我們是每兩個月一次
今年我們就是偶數月
所以應該是在6月底之前
但是議會的關係
我們原來是排在上上禮拜
那勉強有過半數
但是臨時剛好
市長又有請那個國風局長
跟他一起去一個行程
所以我們那天就變成
會員人數就不

In [10]:
# 將輸出會議逐字稿和會議 agenda 組合成 Prompt  

agenda = '''
會議名稱: 新北市住都中心第二屆第四次董監事會
日期: 2024/07/04 14:00-16:00
地點: 新北市新店區公所住都中心辦公室
出席人員: 董事: 9人 , 監事: 2人

會議主要內容: 報告案 (4項), 討論案 (4項), 臨時動議

會議時長: 約2小時 (根據主持人最後的發言推測)

1 報告案
1.1 上次會議決議執行情形說明
1.2 本中心113年5-6月重要業務報告
1.3 本中心113年1-4月預算執行情形報告
1.4 本中心內部稽核業務報告
2 討論案  
2.1 本中心內部稽核人員聘任追認
2.2 本中心114年度業務及營運計畫
2.3 本中心114年度預算案
2.4 本中心擔任台北市文山區木柵段一小段912-1號等24筆土地公辦都市更新實施者案
3 臨時動議 

'''


meeting_minutes_prompt = f'''
你是一位專業的會議記錄助理。你的任務是接續生成會議紀錄,請注意以下幾點:
   1. 仔細閱會議逐字稿、Agenda。
   2. 不要重複已輸出的內容。
   3. 遵循固定結構和格式,保持一致性。特別注意:
   - 明確區分"報告案"和"討論案"部分
   - 保留原有的案由編號(如"案由一"、"案由二"等)
   - 使用適當的格式(標題、子標題、項目符號等)和排版(加粗、縮進等)
   
   架構參考 : 
   ####
    ex. 案由編號:[標題]
    董監事意見:
      - [董監事對於此案由意見列表]
    中心說明:
     - [中心對於此案由說明]。
    決議:[決議內容]
   ####
   
   4. 確保包含以下元素:
      a) 詳細記錄每個議題的討論內容
      b) 突出主要決策和結論
      c) 記錄董監事意見和中心的回應
      d) 使用"決議:"標記每個議題的最終決定
      e) 列出行動項目(如有),包括負責人和期限
   5. 只基於逐字稿中的事實,對不確定的資訊使用[待定]標記。
   6. 使用清晰、專業的繁體中文。
   7. 注意捕捉重要數據和統計資料。
   9. 當你完成所有議程項目，請加上"[會議結束]"標記。

請記住,你的產出對於工作效率和決策執行至關重要,務必謹慎、準確地完成任務。
####
逐字稿:
{transcribed_text}
####
Agenda :
{agenda}
會議紀錄 : 
'''

In [3]:
### 呼叫大語言模型生成會議紀錄 
### 目前輸出字數限制 8192 : 長時間的會議紀錄會超過 Token 限制
#### 新增模型判斷目前的會議紀錄是否完成：如果 Y 則完成輸出會議紀錄，如果 N 則組合 Prompt 前面加請接續生成 Prompt，後面加上目前的會議紀錄，並將結果與之前的輸出組合，直到模型判斷完成輸出會議紀錄

from google.cloud import aiplatform
import vertexai
from vertexai.preview.generative_models import GenerativeModel, SafetySetting

project_id="nthurc-aisearch-202406"
region="asia-east1"

vertexai.init(project=project_id, location=region)
aiplatform.init(project=project_id, location=region)
model = GenerativeModel(
    "gemini-1.5-pro",
  )

generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0.2,
    "top_p": 1,
    "top_k": 1,
}

generation_config_conditional = {
    "max_output_tokens": 1,
    "temperature": 0.0,
    "top_p": 1,
    "top_k": 1,
}

safety_settings = [
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=SafetySetting.HarmBlockThreshold.BLOCK_NONE
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=SafetySetting.HarmBlockThreshold.BLOCK_NONE
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=SafetySetting.HarmBlockThreshold.BLOCK_NONE
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=SafetySetting.HarmBlockThreshold.BLOCK_NONE
    ),
]

# 初始化會議紀錄
meeting_minutes = ""
count = 0

while True:
    # 如果是第一次生成，使用原始的 prompt
    if not meeting_minutes:
        current_prompt = meeting_minutes_prompt
    else:
        # 如果不是第一次，則在原始 prompt 前加上請求繼續生成的指示
        current_prompt = f"""
        你是一位專業的會議記錄助理。你的任務是接續生成會議紀錄,請注意以下幾點:
        1. 仔細閱讀之前提供的會議逐字稿、Agenda和已生成的部分會議紀錄。
        2. 從上次中斷的地方繼續生成,不要重複已經輸出的內容。
        3. 遵循之前的結構和格式,保持一致性。特別注意:
        - 明確區分"報告案"和"討論案"部分
        - 保留原有的案由編號(如"案由一"、"案由二"等)
        - 使用適當的格式(標題、子標題、項目符號等)和排版(加粗、縮進等)
        
        架構參考 : 
        ####
            ex. 案由編號:[標題]
            董監事意見:
            - [董監事對於此案由意見列表]
            中心說明:
            - [中心對於此案由說明]。
            決議:[決議內容]
        ####
        
        4. 確保包含以下元素:
            a) 詳細記錄每個議題的討論內容
            b) 突出主要決策和結論
            c) 記錄董監事意見和中心的回應
            d) 使用"決議:"標記每個議題的最終決定
            e) 列出行動項目(如有),包括負責人和期限
        5. 只基於逐字稿中的事實,對不確定的資訊使用[待定]標記。
        6. 使用清晰、專業的繁體中文。
        7. 注意捕捉重要數據和統計資料。
        8. 如果尚未完成,請在會議紀錄最後提供簡潔的會議摘要。
        9. 當你完成所有議程項目，請加上"[會議結束]"標記。

        請直接從上次中斷的地方繼續生成,無需重複介紹或總結已完成的部分。你的任務是完成整個會議紀錄,直到涵蓋所有議程項目為止。

        ####
        逐字稿:
        {transcribed_text}
        ####
        Agenda:
        {agenda}
        ####
        已生成的會議紀錄:
        {meeting_minutes}
        ####
        
        請從上次中斷的地方繼續生成會議紀錄。
        
        
        """
        count += 1 

    # 生成會議紀錄
    print(f"开始第 {count + 1} 次生成")
    response = model.generate_content(
        contents=current_prompt,
        generation_config=generation_config,
        safety_settings=safety_settings,
    )

    # 更新會議紀錄
    meeting_minutes += response.text
    
    print(f"當前會議紀錄長度: {len(meeting_minutes)} 個字")

    # 檢查是否完成
    
    needContinue_prompt = f'''
    你是一位專業的會議紀錄審核員。你的任務是根據提供的 Agenda 和會議紀錄內容，判斷會議紀錄是否完整完成。請仔細閱讀會議紀錄，並根據以下標準進行評估：

        1. 會議基本信息是否完整（日期、時間、地點、參與者等）
        2. 是否包含所有 Agenda 中列出的議題
        3. 每個議題是否有詳細的討論內容記錄
        4. 是否記錄了董監事的意見和中心的回應
        5. 每個議題是否有明確的決議
        6. 是否列出了行動項目（如有）
        7. 是否有一個總結性的會議摘要
        8. 會議紀錄是否以 "[會議結束]" 標記結束

    評估完成後，請根據以下規則輸出結果：
    - 如果會議紀錄滿足上述所有標準，並且以 "[會議結束]" 標記結束，請輸出 "Y"
    - 如果會議紀錄缺少任何一項上述內容，或者沒有 "[會議結束]" 標記，請輸出 "N"

    請只輸出 "Y" 或 "N"，不需要其他解釋或評論。
    ####

    會議 Agenda : {agenda}

    ####

    會議逐字稿 : {transcribed_text} 

    ####

    目前會議紀錄 : {meeting_minutes}
    '''
    
    continue_response = model.generate_content(
        contents=needContinue_prompt,
        generation_config=generation_config_conditional,
        safety_settings=safety_settings,
    )
    
    print(f"第 {count + 1} 次生成是否完成: {continue_response.text.strip().upper()}")

    # 如果完成，則跳出循環
    if continue_response.text.strip().upper() == "Y":
        break
    
    if count >= 4 :
        break



开始第 1 次生成
當前會議紀錄長度: 2620 個字
第 1 次生成是否完成: N
开始第 2 次生成
當前會議紀錄長度: 8469 個字
第 2 次生成是否完成: Y


In [4]:
print(meeting_minutes)

## **新北市住宅及都市更新中心第二屆第四次董監事會 會議紀錄**

**時間:** 中華民國113年7月4日(星期二)下午2時0分至4時0分

**地點:** 新北市新店區公所住都中心辦公室

**主席:** 陳董事長純敬                **紀錄:**  [待定]

**出席:** 
* 董事: 9人
* 監事: 2人

**列席:** 各部門主管

**壹、報告案**

**案由三、本中心113年1-4月預算執行情形報告**

**說明:**

* 本報告案由行政部財務組組長吳妙芬報告，說明本中心113年1-4月預算執行情形。
    * 1-4月收入執行率為99%，執行數為1億1,600萬1,000元。
    * 成本及費用執行率為71%，執行數為4,460萬5,000元，剩餘數為7,139萬6,000元。
    * 業務收入執行率為98%，主要為社宅租金及權利金收入。
    * 其中，財政局委託管理的社宅及出租宅執行率為41%，因1-4月貸款社宅採用「收付實現制」列帳，故未認列社會住宅收入，產生預決算差異數。
    * 經與會計師討論後，5月份起將依照預算編列原則，調整為「收付實現制」與「權責發生制」並列方式表達，未來報表將呈現收入與支出金額。
    * 若將民權社宅收入計入，1-4月業務收入執行率可提高至103%。
    * 公辦社宅住宅收入整體執行率為105%，既有社宅(相次1-7)執行率皆在95%以上，新接管社宅(相次8-12)預計8月份開始陸續點交招租，故1-4月尚無預算分配數。
    * 榮獎社會住宅租金1-4月執行率為67%，落差原因說明如下：
        * 三峽北大社宅(相次1)執行率為79%，因部分房舍漏水修繕，期間無租金收入。
        * 永和國光社宅(相次2)於2月份點交予社會局管理，預算僅編列1-2月，執行率為53%。
        * 五股城州1號社宅(相次9)執行率為0%，配合招租作業期程，預計8月份開始招租入住。
        * 新接管社宅(相次10-14)因4月底前仍在業務點交及裝修作業中，執行率為0%，後續將配合招租期程開始招租入住。
        * 若排除受建商延遲點交影響的社宅(相次10-14)，執行率可回升至99%。
    * 成本及費用執行方面，1-4月執行率為

In [5]:
def clean_meeting_minutes(meeting_minutes):
    # 查找 "[會議結束]" 标记的位置
    end_tag_index = meeting_minutes.find("[會議結束]")
    
    if end_tag_index != -1:
        # 如果找到标记，只保留标记之前的内容
        cleaned_minutes = meeting_minutes[:end_tag_index].strip()
    else:
        # 如果没有找到标记，保留原始内容
        cleaned_minutes = meeting_minutes
    
    return cleaned_minutes

cleaned_minutes = clean_meeting_minutes(meeting_minutes)
print(cleaned_minutes)

## **新北市住宅及都市更新中心第二屆第四次董監事會 會議紀錄**

**時間:** 中華民國113年7月4日(星期二)下午2時0分至4時0分

**地點:** 新北市新店區公所住都中心辦公室

**主席:** 陳董事長純敬                **紀錄:**  [待定]

**出席:** 
* 董事: 9人
* 監事: 2人

**列席:** 各部門主管

**壹、報告案**

**案由三、本中心113年1-4月預算執行情形報告**

**說明:**

* 本報告案由行政部財務組組長吳妙芬報告，說明本中心113年1-4月預算執行情形。
    * 1-4月收入執行率為99%，執行數為1億1,600萬1,000元。
    * 成本及費用執行率為71%，執行數為4,460萬5,000元，剩餘數為7,139萬6,000元。
    * 業務收入執行率為98%，主要為社宅租金及權利金收入。
    * 其中，財政局委託管理的社宅及出租宅執行率為41%，因1-4月貸款社宅採用「收付實現制」列帳，故未認列社會住宅收入，產生預決算差異數。
    * 經與會計師討論後，5月份起將依照預算編列原則，調整為「收付實現制」與「權責發生制」並列方式表達，未來報表將呈現收入與支出金額。
    * 若將民權社宅收入計入，1-4月業務收入執行率可提高至103%。
    * 公辦社宅住宅收入整體執行率為105%，既有社宅(相次1-7)執行率皆在95%以上，新接管社宅(相次8-12)預計8月份開始陸續點交招租，故1-4月尚無預算分配數。
    * 榮獎社會住宅租金1-4月執行率為67%，落差原因說明如下：
        * 三峽北大社宅(相次1)執行率為79%，因部分房舍漏水修繕，期間無租金收入。
        * 永和國光社宅(相次2)於2月份點交予社會局管理，預算僅編列1-2月，執行率為53%。
        * 五股城州1號社宅(相次9)執行率為0%，配合招租作業期程，預計8月份開始招租入住。
        * 新接管社宅(相次10-14)因4月底前仍在業務點交及裝修作業中，執行率為0%，後續將配合招租期程開始招租入住。
        * 若排除受建商延遲點交影響的社宅(相次10-14)，執行率可回升至99%。
    * 成本及費用執行方面，1-4月執行率為

### 測試程式碼輸出是否足夠穩定

In [13]:
from google.cloud import aiplatform
import vertexai
from vertexai.preview.generative_models import GenerativeModel, SafetySetting

def generate_meeting_minutes(transcribed_text, agenda, meeting_minutes_prompt):
    project_id = "nthurc-aisearch-202406"
    region = "asia-east1"
    vertexai.init(project=project_id, location=region)
    aiplatform.init(project=project_id, location=region)
    model = GenerativeModel("gemini-1.5-flash")
    
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 0.2,
        "top_p": 1,
        "top_k": 1,
    }
    generation_config_conditional = {
        "max_output_tokens": 1,
        "temperature": 0.0,
        "top_p": 1,
        "top_k": 1,
    }
    safety_settings = [
        SafetySetting(category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH, threshold=SafetySetting.HarmBlockThreshold.BLOCK_NONE),
        SafetySetting(category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT, threshold=SafetySetting.HarmBlockThreshold.BLOCK_NONE),
        SafetySetting(category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT, threshold=SafetySetting.HarmBlockThreshold.BLOCK_NONE),
        SafetySetting(category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT, threshold=SafetySetting.HarmBlockThreshold.BLOCK_NONE),
    ]

    meeting_minutes = ""
    count = 0
    while True:
        if not meeting_minutes:
            current_prompt = meeting_minutes_prompt
        else:
            current_prompt = f"""
            你是一位專業的會議記錄助理。你的任務是接續生成會議紀錄,請注意以下幾點:
            1. 仔細閱讀之前提供的會議逐字稿、Agenda和已生成的部分會議紀錄。
            2. 從上次中斷的地方繼續生成,不要重複已經輸出的內容。
            3. 遵循之前的結構和格式,保持一致性。特別注意:
            - 明確區分"報告案"和"討論案"部分
            - 保留原有的案由編號(如"案由一"、"案由二"等)
            - 使用適當的格式(標題、子標題、項目符號等)和排版(加粗、縮進等)
            
            架構參考 : 
            ####
                ex. 案由編號:[標題]
                董監事意見:
                - [董監事對於此案由意見列表]
                中心說明:
                - [中心對於此案由說明]。
                決議:[決議內容]
            ####
            
            4. 確保包含以下元素:
            a) 詳細記錄每個議題的討論內容
            b) 突出主要決策和結論
            c) 記錄董監事意見和中心的回應
            d) 使用"決議:"標記每個議題的最終決定
            e) 列出行動項目(如有),包括負責人和期限
            5. 只基於逐字稿中的事實,對不確定的資訊使用[待定]標記。
            6. 使用清晰、專業的繁體中文。
            7. 注意捕捉重要數據和統計資料。
            8. 如果尚未完成,請在會議紀錄最後提供簡潔的會議摘要。
            9. 當你完成所有議程項目，請加上"[會議結束]"標記。
            請直接從上次中斷的地方繼續生成,無需重複介紹或總結已完成的部分。你的任務是完成整個會議紀錄,直到涵蓋所有議程項目為止。
            ####
            逐字稿:
            {transcribed_text}
            ####
            Agenda:
            {agenda}
            ####
            已生成的會議紀錄:
            {meeting_minutes}
            ####
            請從上次中斷的地方繼續生成會議紀錄。
            """
        count += 1
        print(f"开始第 {count} 次生成")
        response = model.generate_content(
            contents=current_prompt,
            generation_config=generation_config,
            safety_settings=safety_settings,
        )
        meeting_minutes += response.text
        print(f"當前會議紀錄長度: {len(meeting_minutes)} 個字")
        
        needContinue_prompt = f'''
        你是一位專業的會議紀錄審核員。你的任務是根據提供的 Agenda 和會議紀錄內容，判斷會議紀錄是否完整完成。請仔細閱讀會議紀錄，並根據以下標準進行評估：
        1. 會議基本信息是否完整（日期、時間、地點、參與者等）
        2. 是否包含所有 Agenda 中列出的議題
        3. 每個議題是否有詳細的討論內容記錄
        4. 是否記錄了董監事的意見和中心的回應
        5. 每個議題是否有明確的決議
        6. 是否列出了行動項目（如有）
        7. 是否有一個總結性的會議摘要
        8. 會議紀錄是否以 "[會議結束]" 標記結束
        評估完成後，請根據以下規則輸出結果：
        - 如果會議紀錄滿足上述所有標準，並且以 "[會議結束]" 標記結束，請輸出 "Y"
        - 如果會議紀錄缺少任何一項上述內容，或者沒有 "[會議結束]" 標記，請輸出 "N"
        請只輸出 "Y" 或 "N"，不需要其他解釋或評論。
        ####
        會議 Agenda : {agenda}
        ####
        會議逐字稿 : {transcribed_text}
        ####
        目前會議紀錄 : {meeting_minutes}
        '''
        continue_response = model.generate_content(
            contents=needContinue_prompt,
            generation_config=generation_config_conditional,
            safety_settings=safety_settings,
        )
        print(f"第 {count} 次生成是否完成: {continue_response.text.strip().upper()}")
        if continue_response.text.strip().upper() == "Y":
            break
        if count >= 4:
            break
    
    return meeting_minutes, continue_response.text.strip().upper() == "Y"

import os

def clean_meeting_minutes(meeting_minutes):
    end_tag_index = meeting_minutes.find("[會議結束]")
    if end_tag_index != -1:
        cleaned_minutes = meeting_minutes[:end_tag_index].strip()
    else:
        cleaned_minutes = meeting_minutes
    return cleaned_minutes

def test_generate_meeting_minutes(transcribed_text, agenda, meeting_minutes_prompt, num_tests=10):
    successful_generations = 0
    generations_with_end_tag = 0

    # 創建一個資料夾來存儲輸出文件
    output_folder = "meeting_minutes_output"
    os.makedirs(output_folder, exist_ok=True)

    for i in range(num_tests):
        print(f"\n開始第 {i+1} 次測試")
        minutes, is_successful = generate_meeting_minutes(transcribed_text, agenda, meeting_minutes_prompt)
        
        # 清理會議紀錄
        cleaned_minutes = clean_meeting_minutes(minutes)
        
        # 保存清理後的會議紀錄到文件
        file_name = f"meeting_minutes_test_{i+1}.md"
        file_path = os.path.join(output_folder, file_name)
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(cleaned_minutes)
        
        if is_successful:
            successful_generations += 1
        
        if "[會議結束]" in minutes:
            generations_with_end_tag += 1
        
        print(f"測試 {i+1} 完成：成功 = {is_successful}, 包含 [會議結束] 標記 = {'[會議結束]' in minutes}")
        print(f"清理後的會議紀錄已保存到: {file_path}")

    print(f"\n總結：")
    print(f"成功生成次數: {successful_generations} / {num_tests}")
    print(f"包含 [會議結束] 標記的次數: {generations_with_end_tag} / {num_tests}")
    print(f"所有清理後的會議紀錄已保存到 '{output_folder}' 資料夾")

test_generate_meeting_minutes(transcribed_text, agenda, meeting_minutes_prompt)

ModuleNotFoundError: No module named 'sklearn'